In [5]:
from typing import *

import numpy as np
from numpy.random import uniform

In [8]:
class HyperParameters:
    def __init__(self, 
                 num_params:int, 
                 limits: list[tuple[float,float]], 
                 parameters: np.ndarray[float]=None):
        if not isinstance(num_params, int):
            raise TypeError('num_params must be an integer.')
        if num_params <= 0:
            raise ValueError('num_params must be greater than 0.')
        if len(limits) != num_params:
            raise ValueError('list of limits should be of length "num_params". ')
        if parameters is not None:
            if len(parameters) != len(limits):
                raise ValueError('When explicitly provided, the number of parameters should be equal to "num_params". ')
        self.num_params = num_params
        self.limits = limits
        self.parameters = parameters
        
    def get_parameters(self):
        if self.parameters is None:
            raise RuntimeError(f' "parameters" attribute has not been set. ')
        return self.parameters
    
    def set_parameters(self, parameters: np.ndarray[float]):
        if len(parameters) != len(self.limits):
                raise ValueError('When explicitly provided, the number of parameters should be equal to "num_params". ')
        self.parameters = parameters
    
    def get_random_parameters(self, distribution: str = 'uniform'):
        if distribution == 'uniform':
            self.parameters = np.array([uniform(low=limit[0], high=limit[1], size=1) for limit in self.limits])
            return self.get_parameters()
        

class CovarianceMatrix:
    def __init__(self, kernel: Callable):
        self.kernel = kernel
        self.matrix = None
        
    def set_matrix(self, samples: list[np.ndarray[float]]):
        N = len(samples)
        matrix = np.empty((N, N),dtype=float)
        for i in range(N):
            for j in range(i,N):
                matrix[i,j] = self.kernel(samples[i],samples[j])
                if i != j:
                    matrix[j, i] = matrix[i, j]
        self.matrix = matrix
    
    def get_matrix(self):
        if self.matrix is None:
            raise RuntimeError(f' "matrix" attribute has not yet been set. ')
        return self.matrix
    
    
def rbf_kernel(x1, x2, length_scale=1.0, sigma_f=1.0):
    """
    Compute the Radial Basis Function (RBF) kernel between two d-dimensional vectors.

    :param x1: First d-dimensional input vector.
    :param x2: Second d-dimensional input vector.
    :param length_scale: The length scale of the kernel.
    :param sigma_f: The signal variance (amplitude) of the kernel.
    :return: Scalar value representing the covariance between x1 and x2.
    """
    # Ensure the inputs are arrays (in case they are lists or tuples)
    x1 = np.asarray(x1)
    x2 = np.asarray(x2)
    
    # Compute the squared Euclidean distance between the vectors
    sqdist = np.sum((x1 - x2)**2)
    
    # Compute the RBF kernel
    return sigma_f**2 * np.exp(-0.5 / length_scale**2 * sqdist)  
        
        
    
def rosenbrock(x: float, y: float) -> float:
    return (1 - x) ** 2 + 100 * (y-x ** 2) ** 2

In [9]:
a, b = np.array([1,2]), np.array([3,2])
rbf_kernel(a,b)

0.1353352832366127